# 线性回归实现波士顿房价预测

In [ ]:
import socket
hostname =socket.gethostname()
ip = socket.gethostbyname(hostname)
print("本机IP地址：",ip)

## 一、导入数据

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

In [ ]:
boston = fetch_openml(name='boston')
X = pd.DataFrame(boston.data, columns=boston.feature_names) 
y = pd.DataFrame(boston.target, columns=["MEDV"])

X

## 二、数据分析

In [ ]:
X.shape, y.shape

In [ ]:
X.info()

In [ ]:
X. describe()

In [ ]:
X.isnull().sum()

In [ ]:
df = pd.concat([X, y], axis=1)
df_corr = df.corr()

In [ ]:
import seaborn as sns

sns.heatmap(df_corr, cmap = "coolwarm", annot = True)

## 三、数据预处理

In [ ]:
# 数据标准化
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 转换为张量
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
y_train =torch. squeeze(torch. from_numpy(y_train. values)). float()
y_test = torch.squeeze(torch.from_numpy(y_test.values)).float()

## 四、构建模型

### 构建线性回归模型

In [ ]:
class LinearRegression(nn.Module): #定义模型类型
    def __init__(self, input_dim):
        super(LinearRegression, self).__init__() #固定写法
        self.linear = nn.Linear(input_dim, 1) #使用torch.nn中的线性回归模型
#组建模型框架
    def forward(self, x):#模型向前传播
        out = self.linear(x) 
        return out

### 训练模型

In [ ]:
# 定义模型
input_dim = X_train.shape[1]
# 实例化模型
model = LinearRegression(input_dim)
# 定义损失函数和优化器
criterion = nn.MSELoss() #均方误差
optimizer = torch.optim.SGD(model.parameters(), lr=0.003) #随机 梯度下降
# 训练模型
num_epochs = 1000 #训练1000次
losses = [] #记录每100次训练方差 
for epoch in range(num_epochs):
# 前向传播得到模型输出 
    y_pred = model(X_train) 
    # 计算均方误差
    mse = criterion(y_pred, y_train)
# 反向传播和优化
    optimizer. zero_grad()
    mse.backward() #反向传播，计算模型梯度 
    optimizer.step() #更新模型参数
# 打印日志
    if (epoch+1) % 100 == 0:
        losses. append(mse. item())
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1,num_epochs, mse.item()))

### 训练方差变换趋势图

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(losses, label="train_MSEL") 
plt. legend()

In [ ]:
# 测试模型
with torch.no_grad():
    y_pred = model(X_test)
    mse = criterion(y_pred, y_test)
    print('Test Loss: {:.4f}'.format(mse.item()))

In [ ]:
plt. plot(y_pred,color= "r") 
plt. plot(X_test,color= "b")